In [1]:
#Esempio di classificazione Binaria utilizzata per rilevare un attacco SYN-DOS 

In [2]:
from pyspark.sql import SparkSession
import time
#from pyspark.sql.functions import lit

#Inizialize Spark Context
spark = SparkSession.builder.appName("SYN-DOS").getOrCreate()
sc = spark.sparkContext

In [3]:
#Load Training Set
#Q1: Cosa succede se non usiamo inferschema
syn_dataset=spark.read.option("header", "false").option("inferschema","true").csv("/FileStore/tables/syn_dos_11000.csv")

In [4]:
syn_dataset.printSchema()

In [5]:
#syn_dataset.printSchema()
syn_dataset = syn_dataset.withColumnRenamed("_c115","Class")
syn_dataset.printSchema()
#syn_dataset.head()

In [6]:
print("Pos instances: ", syn_dataset.filter("Class='T'").count())
print("Neg instances: ", syn_dataset.filter("Class='F'").count())

In [7]:
#Create vector of Features & Label, conpacting all feature in a Vector and converting Class from String to Double
from pyspark.ml.feature import RFormula

assemblerFormula = RFormula(formula="Class ~ .")
trainingTF = assemblerFormula.fit(syn_dataset)
syn_dataset_prepared = trainingTF.transform(syn_dataset).select("features", "label")
syn_dataset_prepared.printSchema()
#Spark-ML algorithms requires a single vector containing each features
#Assemble vector of features
#assembler = VectorAssembler(inputCols=training_set.columns[0:len(training_set.columns)-1],outputCol="features")
#training=assembler.transform(training_set).select("label","features")

#syn_dataset_prepared.count() #
#syn_dataset_prepared.filter("Class='F'").count()  #T=7038 # F=90000

In [8]:
syn_dataset_prepared.show()

In [9]:
#Create Training set and Test set

training, test = syn_dataset_prepared.randomSplit([70.0,30.0],seed=13)
training.cache()
test.cache()
#training.count()
print("Training Tot instances: ", training.count())
print("Test Tot instances: ", test.count())

In [10]:
#LOGISTIC REGRESSION
alg_label="LOGISTIC REGRESSION"
from pyspark.ml.classification import LogisticRegression
start_time = time.time()
lr_model = LogisticRegression()
lr_fitted = lr_model.fit(training)
print("%s Training time: %5.2f seconds ---" % (alg_label,(time.time() - start_time)))
#LOGISTIC REGRESSION  TEST
start_time = time.time()
predictions_and_labels = lr_fitted.evaluate(test)
print("%s Test: %5.2f seconds ---" % (alg_label,(time.time() - start_time)))
#Evaluation
#cm=classification_evaluation(predictions_and_labels.predictions.select( "label", "prediction"),label_to_print=alg_label)
#Accuracy
predictions_and_labels.accuracy


In [11]:
predictions_and_labels.predictions.filter("label=prediction").count()

In [12]:
#Decision TREE CLASSIFIER
alg_label="DECISION TREE CLASSIFIER"
from pyspark.ml.classification import DecisionTreeClassifier
start_time = time.time()
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features",maxDepth=4)
dt_fitted = dt.fit(training)
print("%s Training time: %5.2f seconds ---" % (alg_label,(time.time() - start_time)))
start_time = time.time()
dt_predictions_and_labels = dt_fitted.transform(test)


print("%s Test: %5.2f seconds ---" % (alg_label,(time.time() - start_time)))

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
print ("Accuracy: %3.2f%%" % (evaluator.evaluate(dt_predictions_and_labels)*100))



In [13]:
#print(dt.explainParams())
#Features Importance 
print(dt_fitted.featureImportances)
#Explicit print of decision Tree
print(dt_fitted.toDebugString)

In [14]:
#How to save and load a trained model
from pyspark.ml.classification import DecisionTreeClassificationModel

dt_fitted.save("/FileStore/tables/dt-syndos-tra")
dt_persisted = DecisionTreeClassificationModel.load("/FileStore/tables/dt-syndos-trained_1")
dt_persisted.transform(test).show()
#dt_fitted

In [15]:
#Use SQL to query a Data Frame
dt_predictions_and_labels.createOrReplaceTempView("treePredictions")


In [16]:
%sql
select label, count(label) from treePredictions group by label

In [17]:
#SPARK Documentation for Random Forest:
#https://spark.apache.org/docs/2.2.0/ml-classification-regression.html#random-forest-classifier

#Ramdom Forest Classifier TRAINING
alg_label="RANDOM FOREST CLASSIFIER"
from pyspark.ml.classification import RandomForestClassifier
start_time = time.time()
rf = RandomForestClassifier(labelCol="label", featuresCol="features",numTrees=100, maxDepth=15)
rf_fitted = rf.fit(training)
print("%s Training time: %5.2f seconds ---" % (alg_label,(time.time() - start_time)))
start_time = time.time()
rf_predictions_and_labels = rf_fitted.transform(test)
print("%s Test: %5.2f seconds ---" % (alg_label,(time.time() - start_time)))
#Evaluation    
#cm=classification_evaluation(rf_predictions_and_labels,label_to_print=alg_label)

In [18]:
#Accuracy
rf_predictions_and_labels.filter("label=prediction").count()/rf_predictions_and_labels.count()
#rf_fitted.featureImportances
#rf_fitted.toDebugString


In [19]:
#Multilayer Perceptron Classifier
alg_label="Perceptron Classifier"

from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
start_time = time.time()
# specify layers for the neural network:
# input layer of size 115 (Features)
# and output of size 2 (classes)
layers = [115, 90, 2]

# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)

# train the model
model = trainer.fit(training)
print("%s Training time: %5.2f seconds ---" % (alg_label,(time.time() - start_time)))
# compute accuracy on the test set
start_time = time.time()
result = model.transform(test)
print("%s Test: %5.2f seconds ---" % (alg_label,(time.time() - start_time)))
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Accuracy = " + str(evaluator.evaluate(predictionAndLabels)))


In [20]:
#https://people.apache.org/~pwendell/spark-nightly/spark-master-docs/latest/ml-tuning.html
#K-FOLD VALIDATION

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# We now treat the Pipeline as an Estimator, wrapping it in a CrossValidator instance.
# This will allow us to jointly choose parameters for all Pipeline stages.
# A CrossValidator requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
# We use a ParamGridBuilder to construct a grid of parameters to search over.
# With 3 values for hashingTF.numFeatures and 2 values for lr.regParam,
# this grid will have 3 x 2 = 6 parameter settings for CrossValidator to choose from.

start_time = time.time()

#pipeline = Pipeline(stages=[assemblerFormula, rf])
pipeline = Pipeline(stages=[rf])
paramGrid = ParamGridBuilder().build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=3)  # use 3+ folds in practice

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(training)

print("%s Training time: %5.2f seconds ---" % ("3-fold cross validation",(time.time() - start_time)))

# Make predictions on test documents. cvModel uses the best model found.
prediction = cvModel.transform(test)

selected = prediction.select("probability", "prediction")

In [21]:
#Wich parameters for fine tuning ?
print(dt.explainParams())

In [22]:
#https://people.apache.org/~pwendell/spark-nightly/spark-master-docs/latest/ml-tuning.html
#K-FOLD VALIDATION + Parameter Tuning

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# We now treat the Pipeline as an Estimator, wrapping it in a CrossValidator instance.
# This will allow us to jointly choose parameters for all Pipeline stages.
# A CrossValidator requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
# We use a ParamGridBuilder to construct a grid of parameters to search over.
# With 3 values for hashingTF.numFeatures and 2 values for lr.regParam,
# this grid will have 3 x 2 = 6 parameter settings for CrossValidator to choose from.

start_time = time.time()

pipeline = Pipeline(stages=[rf])
paramGrid = ParamGridBuilder() \
    .addGrid(rf.maxDepth, [3, 5, 15]) \
    .addGrid(rf.maxBins, [5, 32, 50]) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=3)  # use 3+ folds in practice

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(training)

print("%s Training time: %5.2f seconds ---" % ("3-fold cross validation",(time.time() - start_time)))

# Make predictions on test documents. cvModel uses the best model found.
prediction = cvModel.transform(test)

#Get manually the best model
cvModel.bestModel.transform(test)

selected = prediction.select("probability", "prediction")

In [23]:
#BONUS
#KMeans
from pyspark.ml.clustering import KMeans

# Loads data.
#dataset=spark.read.option("header", "false").option("inferschema","true").csv("/FileStore/tables/syn_dos_11000-06df8.csv")
dataset = training.union(test)
# Trains a k-means model.
kmeans = KMeans().setK(2).setSeed(1)
model = kmeans.fit(dataset)

# Evaluate clustering by computing Within Set Sum of Squared Errors.
wssse = model.computeCost(dataset)
print("Within Set Sum of Squared Errors = " + str(wssse))

# Shows the result.
centers = model.clusterCenters()
#Number of dimentions = number of features
print("Cluster Centers: ")
for center in centers:
    print(center)